In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import resnet50



In [2]:
# Подготовка предобученной сети ResNet50
resnet_model = resnet50(weights='DEFAULT')

C:\Users\chest\PycharmProjects\waifu_recommendations\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\chest\PycharmProjects\waifu_recommendations\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\chest/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%


In [3]:
# Удаляем последний слой классификации, оставляя только слои для экстракции признаков
features_extractor = torch.nn.Sequential(*list(resnet_model.children())[:-1])

In [4]:
# Функция предварительной обработки изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Преобразуем размеры изображений
    transforms.ToTensor(),                # Конвертируем в Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   # Нормализуем
])

In [5]:
# Функция для извлечения эмбеддингов из изображения
def extract_features(image_path):
    with torch.no_grad():
        # Загружаем изображение
        image = Image.open(image_path)

        # Применяем преобразования
        transformed_image = transform(image).unsqueeze(0)

        # Экстрагируем эмбеддинги
        embedding = features_extractor(transformed_image)

        # Приводим размерность к одномерному вектору
        flattened_embedding = embedding.view(-1)

    return flattened_embedding.numpy()

In [7]:
images = os.listdir('waifus/images/')


all_names = []
all_vectors = None
features_extractor.eval()
root = 'waifus/images/'


for i, file in enumerate(images):
    img = Image.open(root + file)
    transformed_image = transform(img).unsqueeze(0)

    # Экстрагируем эмбеддинги
    embedding = features_extractor(transformed_image)

    # Приводим размерность к одномерному вектору
    flattened_embedding = embedding.view(-1)

    if all_vectors is None:
        all_vectors = flattened_embedding
    else:
        all_vectors = np.concatenate((all_vectors, flattened_embedding), axis=0)
    all_names.append(file)




(2048,)


In [ ]:
np.save('all_vectors.npy', all_vectors)
np.save('all_names.npy', all_names)